In [14]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random

np.random.seed(42)

In [10]:
# Generating synthetic dataset
# y = m_k*X_k + b_k + noise -> for simplicity generating sythetic dataset for just 1 variable, m = 2, b = 3

samples = 5000
b = 3
m = 2
X = np.random.rand(samples)
noise = np.random.normal(loc=0, scale=1, size=samples)
Y = (m*X) + 3 + noise
Y

array([3.27327705, 3.38477009, 5.21745238, ..., 3.94490117, 4.01943987,
       4.40109145], shape=(5000,))

In [24]:
# Batch Gradient descent

epochs = 1000
lr = 0.01
m = random.random()
b = random.random()

for i in range(epochs):
    
    #calculate y_hat
    y_hat = (m * X) + b
    
    #calculate the MSE
    mse = np.sum(np.pow((y_hat - Y), 2))/(Y.shape[0])
    
    #calculate partial derivates wrt m and b
    partial_der_m = (-2 * np.sum(X * (y_hat - Y))) / (Y.shape[0])
    partial_der_b = (-2 * np.sum(Y - y_hat)) / (Y.shape[0])
    
    # update coefficeints m and b
    m = m - (lr * partial_der_m)
    b = b - (lr * partial_der_b)